# **Multi-GPU Showcase with `persist`**  

**Author:** [Severin Dicks](https://github.com/Intron7)
**Copyright:** [scverse](https://scverse.org)

## **Overview**  

In this notebook, we showcase the **multi-GPU computation capabilities** of **rapids-singlecell**  
using **Dask**, enabling the analysis of **11 million cells at unprecedented speed**.  

This notebook is run on a **DGX system with 8 NVIDIA H100 GPUs**, demonstrating how **Dask** can efficiently distribute computations across multiple GPUs.  

### **Key Advantages of Multi-GPU Computation**  
By leveraging **Dask and RAPIDS**, we can:  
- **Process massive single-cell datasets** without exceeding memory limits.  
- **Fully utilize all available GPUs**, scaling performance across multiple devices.  
- **Enable chunk-based execution**, efficiently managing memory by loading only necessary data.  

### **Combining Multi-GPU with Out-of-Core Processing**  
- **Multi-GPU processing and out-of-core execution** can be combined to analyze even larger datasets that exceed GPU memory.  
- However, in this notebook, **we focus purely on multi-GPU scaling** without out-of-core execution.  

This approach significantly accelerates **large-scale single-cell analysis**,  
making it feasible on high-performance hardware like **DGX systems**,  
while also being adaptable to multi-GPU workstations.

In [1]:
import dask
import time


from dask_cuda import LocalCUDACluster
from dask.distributed import Client

## **Initializing a Multi-GPU Dask Cluster for RAPIDS**  

To fully utilize all **8 NVIDIA H100 GPUs** on the DGX system,  
we initialize a **multi-GPU Dask cluster** and configure **RAPIDS Memory Manager (RMM)**  
for efficient memory handling across GPUs.  

### **Setting Up Memory Management with RMM**  
RAPIDS **RMM (RAPIDS Memory Manager)** helps optimize GPU memory usage by enabling **managed memory**,  
which improves memory efficiency when working with large-scale datasets.

### **Launching a Multi-GPU Dask Cluster**
We create a Dask CUDA cluster that utilizes all 8 GPUs for preprocessing and analysis.

* `rmm.reinitialize(managed_memory=True)` → Enables unified memory management,
allowing GPUs to handle memory allocation more dynamically.
* `cp.cuda.set_allocator(rmm_cupy_allocator)` → Sets CuPy to use the RAPIDS memory allocator.
* `LocalCUDACluster(CUDA_VISIBLE_DEVICES=preprocessing_gpus)`→
Launches a Dask cluster across all available GPUs.
* `client.run(set_mem)` → Configures memory management on all worker nodes.

In [2]:
%%time
import rmm
import cupy as cp

from rmm.allocators.cupy import rmm_cupy_allocator

def set_mem():
    rmm.reinitialize(managed_memory=True)
    cp.cuda.set_allocator(rmm_cupy_allocator)

preprocessing_gpus="0,1,2,3,4,5,6,7"
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=preprocessing_gpus)

client = Client(cluster)

client.run(set_mem)

client

CPU times: user 857 ms, sys: 486 ms, total: 1.34 s
Wall time: 33.1 s


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 1.75 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42671,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 1.75 TiB
Comm: tcp://127.0.0.1:39931,Total threads: 1
Dashboard: http://127.0.0.1:35693/status,Memory: 224.18 GiB
Nanny: tcp://127.0.0.1:46747,


In [3]:
import rapids_singlecell as rsc
import anndata as ad

## **Loading Large Datasets into AnnData with Dask**  

To efficiently handle large-scale single-cell datasets, we load data directly from an **HDF5 (`h5`) or Zarr file**  
into an **AnnData object** using **Dask arrays**. This enables **lazy loading**, allowing data to be processed in chunks  
without exceeding memory limits.  

We achieve this using **`read_elem_as_dask`**, which loads the expression matrix (`X`) as a **Dask array**

In [4]:
from packaging.version import parse as parse_version

if parse_version(ad.__version__) < parse_version("0.12.0rc1"):
    from anndata.experimental import read_elem_as_dask as read_dask
else:
    from anndata.experimental import read_elem_lazy as read_dask

import zarr

SPARSE_CHUNK_SIZE = 20_000
data_pth = "zarr/cell_atlas.zarr" #11Million Cells
#data_pth = "zarr/nvidia_1.3M.zarr" #1.3Million Cells

f = zarr.open(data_pth)
X = f["X"]
shape = X.attrs["shape"]
adata = ad.AnnData(
    X = read_dask(X, (SPARSE_CHUNK_SIZE, shape[1])),
    obs = ad.io.read_elem(f["obs"]),
    var = ad.io.read_elem(f["var"])
)


## **Transferring AnnData to GPU and Persisting Data**  

To leverage **multi-GPU acceleration**, we transfer the AnnData object to GPU memory  
and persist its **Dask-backed expression matrix** for efficient computation. 

**Step-by-Step Breakdown:**
1. Move AnnData to GPU → `rsc.get.anndata_to_GPU(adata)`
    * Transfers all numerical data (`.X`) to GPU memory.
2. Persist the Expression Matrix → `adata.X = adata.X.persist()`
    * Keeps `adata.X` in memory across Dask workers, avoiding redundant recomputation.
3. Optimize Chunking → `adata.X.compute_chunk_sizes()`
    * Computes the exact chunk sizes for optimal Dask scheduling and memory usage.

In [5]:
%%time
rsc.get.anndata_to_GPU(adata)
adata.X = adata.X.persist()
adata.X.compute_chunk_sizes()

CPU times: user 6.85 s, sys: 3.28 s, total: 10.1 s
Wall time: 1min 23s


dask.array<X_to_GPU, shape=(11441407, 45854), dtype=float32, chunksize=(20000, 45854), chunktype=cupyx.csr_matrix>

## **Quality Control (QC) Metrics Calculation**  

Before proceeding with further analysis, we compute **quality control (QC) metrics**  
to assess dataset quality and filter out low-quality cells or genes.  

We use **`rsc.pp.calculate_qc_metrics()`** to calculate key QC metrics

Although we are working with Dask-backed AnnData, this operation requires a synchronization step.
This means that Dask computations must be evaluated immediately,
so the process is not completely lazy like other out-of-core operations.

In [6]:
%%time
rsc.pp.calculate_qc_metrics(adata)

CPU times: user 2.93 s, sys: 562 ms, total: 3.49 s
Wall time: 18.1 s


## **Filtering Cells and Genes Without Additional Computation**  

Instead of using **`sc.pp.filter_cells`** and **`sc.pp.filter_genes`**,  
we apply filtering directly using boolean indexing to **avoid extra computation**.

**Why Use Direct Indexing Instead of Built-in Functions?**
* More Efficient with Dask → Avoids triggering additional computations.
* Preserves Lazy Execution → Filtering is applied without forcing full dataset evaluation.
* Copy is preferred → Using `.copy()` prevents views. Views slow down computations with Dask-backed AnnData.

In [7]:
%%time
adata = adata[(adata.obs["n_genes_by_counts"]<=10000) 
            & (adata.obs["n_genes_by_counts"]>=200)].copy()
adata = adata[:,adata.var["n_cells_by_counts"]>=10].copy()

CPU times: user 20.4 s, sys: 3.58 s, total: 23.9 s
Wall time: 23.1 s


## **Persisting and Optimizing Chunk Sizes After QC and Subsetting**  

After performing **quality control (QC) and subsetting** the dataset,  
we persist the **Dask-backed expression matrix** and optimize its chunk sizes for efficient multi-GPU execution.  
Persisting after filtering ensures that only high-quality, relevant data remains in memory.

In [8]:
%%time
adata.X = adata.X.persist()
adata.X.compute_chunk_sizes()

/opt/conda/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 22.17 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 9.6 s, sys: 3.18 s, total: 12.8 s
Wall time: 1min 27s


dask.array<getitem, shape=(11441244, 41291), dtype=float32, chunksize=(20120, 41291), chunktype=cupyx.csr_matrix>

## **Log Normalization (Fully Lazy Execution)**  

Next, we apply **log normalization** to scale gene expression values.  
This step ensures that differences in sequencing depth across cells do not dominate downstream analysis.  

In [9]:
%%time
rsc.pp.normalize_total(adata,target_sum = 10000)
rsc.pp.log1p(adata)

CPU times: user 7.11 ms, sys: 328 μs, total: 7.44 ms
Wall time: 5.67 ms


## **Storing the Processed Data in Memory**  

After Log Normalization, we persist the updated expression matrix  
to store the new results in memory for efficient access.  

In [10]:
adata.X = adata.X.persist()
adata.X.compute_chunk_sizes()

dask.array<_calc_log1p, shape=(11441244, 41291), dtype=float32, chunksize=(20120, 41291), chunktype=cupyx.csr_matrix>

## **Selecting Highly Variable Genes**  

To focus on the most informative features, we identify **highly variable genes (HVGs)**  
using the **Cell Ranger** method and subset the dataset accordingly.  

* Copy is preferred → Using `.copy()` prevents views. Views slow down computations with Dask-backed AnnData.

In [11]:
%%time
rsc.pp.highly_variable_genes(adata,n_top_genes=5000, flavor="cell_ranger")
adata = adata[:,adata.var.highly_variable].copy()

CPU times: user 32.9 s, sys: 15.6 ms, total: 32.9 s
Wall time: 32.4 s


## **Rechunking the Expression Matrix for Multi-GPU Execution**  

To optimize performance across **8 GPUs**, we rechunk the expression matrix (`adata.X`)  
so that each GPU processes an equal portion of the dataset.  

In [12]:
n_rows = adata.shape[0]
n_cols = adata.shape[1]
rows_per_worker = (n_rows+8-1)//8
adata.X = adata.X.rechunk((rows_per_worker, n_cols)).persist()

adata.X.compute_chunk_sizes()

dask.array<rechunk-merge, shape=(11441244, 5000), dtype=float32, chunksize=(1430156, 5000), chunktype=cupyx.csr_matrix>

## **Scaling Gene Expression (Requires Synchronization)**  

To standardize gene expression values, we apply **feature scaling**,  
We also `persist` the results to ensure fast accessibility

In [13]:
%%time
rsc.pp.scale(adata, zero_center= False)
adata.X = adata.X.persist()
adata.X.compute_chunk_sizes()

/opt/conda/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 43.67 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 241 ms, sys: 207 ms, total: 448 ms
Wall time: 3.27 s


dask.array<scale_kernel_csr, shape=(11441244, 5000), dtype=float32, chunksize=(1430156, 5000), chunktype=cupyx.csr_matrix>

## **Principal Component Analysis (PCA) on GPU**  

To reduce dimensionality while preserving meaningful variation,  
we perform **Principal Component Analysis (PCA)** using **GPU acceleration**.

Finalizing the Transformation with `.compute()`
    * After computing the principal components, the data remains lazy (Dask CuPy array).
    * Calling `.compute()` on `adata.obsm["X_pca"]` performs the final transformation,
      projecting the data onto the computed PCs and materializing the result as a fully computed CuPy array.

In [14]:
%%time
rsc.pp.pca(adata, n_comps = 100,mask_var=None)
adata.obsm["X_pca"]=adata.obsm["X_pca"].persist()
adata.obsm["X_pca"].compute_chunk_sizes()

CPU times: user 711 ms, sys: 302 ms, total: 1.01 s
Wall time: 6.76 s


dask.array<_transform, shape=(11441244, 100), dtype=float32, chunksize=(1430156, 100), chunktype=cupy.ndarray>